# TP

## 1. Balacing demand and supply
In case the demand is more than supply, then dummy origin is added to the table. The supply of dummy origin will be equal to the difference between the total supply and total demand. The cost associated with the dummy origin will be zero (or other supply/demand-related cost)

## 2. Obtaining the initial feasable solution

Identifying the solution that satisfies the requirements of demand and supply. 

### Northwest Corner Rule

The North-West Corner Rule is a method adopted to compute the initial feasible solution of the transportation problem. The name North-west corner is given to this method because the basic variables are selected from the extreme left corner. Doesn't consider transportation costs.

<img src="https://www.linearprogramming.info/wp-content/uploads/2015/10/Northwest-Corner-Model-solu.gif"/>

The objective function value:
$Z=5*10+10*2+5*7+15*9+5*20+10*18=520$

### Least Cost Method

The Least Cost Method is another method used to obtain the initial feasible solution for the transportation problem. Here, the allocation begins with the cell which has the minimum cost. The lower cost cells are chosen over the higher-cost cell with the objective to have the least cost of transportation.

- In which cell we shall allocate? Generally, the cost where maximum quantity can be assigned should be chosen to obtain the better initial solution

## 3. Testing the degeneration

- If the solution is degenerate, make use of an artificial quantity ($=0$)
- The quantity is assigned to unoccupied cell, which has the minimum transportation cost
- If quantity is inside of loop and negative (its already $0$ and cannot be reduced further), recalculate using another cell

## 4. Testing the optimality of the solution

### Stepping Stone Method

The Stepping Stone Method is used to check the optimality of the initial feasible solution. Thus, the stepping stone method is a procedure for finding the potential of any non-basic variables (empty cells) in terms of the objective function. Through Stepping stone method, we determine that what effect on the transportation cost would be in case one unit is assigned to the empty cell. With the help of this method, we come to know whether the solution is optimal or not.

#### Conditions

- In a closed loop, cells are selected in a sequence such that one cell is unused/unoccupied, and all other cells are used/occupied
- Never go diagonal and you may pass over cells.
- Never visit a cell twice.

#### Steps

1. Closed loops are formed starting in an empty cell; go to an allocated cell, then another and another until returning to empty cell
2. Look at the giving cells around and shift the minimum to the unallocated cell, while maintaining the feasibility constraints of supply and demand
3. Calculate the delta value of the initial and optimized transportation costs
4. Repeat for all unoccupied cells and the best cell improvement is chosen for the change. 
    - Positive quantities mean no improvement (Ddditional costs)
5. When all unallocated (non-basic) cells evaluate to positive quantities, the tableau is optimal.

### Modified Distribution Method (MODI)

The Modified Distribution Method or MODI is an efficient method of checking the optimality of the initial feasible solution. Modification of Stepping-Stone Method with same prerequisites.

#### Steps

1. Calculate the values of $u_i$ and $v_j$ by using the equation $u_i+v_j = c_{ij}$
    - Introduce the variables $u_i$ (rows) and $v_i$ (columns). Set $u$ to $0$ (Convenient rule is to select the ui, which has the largest number of allocations in its row)
    - For each occupied cell calculate $u_i$ and $v_i$ using equation $c_{ij}=u_i+v_i$.
2. Next step is to calculate the opportunity costs (or reduced costs) of the unoccupied cells by using the formula $c_{ij}-u_i+v_i$ (which becomes the same value using Stepping Stone)
3. Choose the largest opportunity cost and start from there
4. Construct a closed path
    - Reallocation is done so that every other cell in the loop remains positive
    - No diagonal movements allowed
    - Maximum two occupied cells from the same row or column can be included in the path
    <img src="https://businessjargons.com/wp-content/uploads/2015/12/MIDI-Final.jpg"/>
5. To test whether the solution is the most optimal, repeat from 3. until all opportunity costs are positive.

#### Usable links

- Solve Transportation Problem online: http://cbom.atozmath.com/CBOM/Transportation.aspx?q=modi
- MODI illustration: http://zoomin.idt.mdh.se/course/kpp227/Documents/MODI.pdf

# TSP
Given a list of cities and the distances between each pair of cities, what is the shortest possible route that visits each city and returns to the origin city?

## Construction Heuristics
A heuristic is a technique designed for solving a problem more quickly when classic methods are too slow.

### Nearest Unvisited Neighbor Heuristic
The nearest neighbour (NN) algorithm (D greedy algorithm) lets the salesman choose the nearest unvisited city as his next move. This algorithm quickly yields an effectively short route. For N cities randomly distributed on a plane, the algorithm on average yields a path 25% longer than the shortest possible path.

In [1]:
import numpy as np

def to_edges(route):
    return list(zip(route, route[1:]))

def length(D, route):
    edges = to_edges(route)
    return np.sum([D[e[0], e[1]] for e in edges])

In [649]:
def NN(D, start):
    route = [start]
    i = start
    while len(route) < len(D[0]):
        min_d = D[i, :].max() # initial value, must be non-zero
        min_j = D[i, :].argmax()
        for j, d in enumerate(D[i, :]):
            if j != i and j not in route and d < min_d:
                min_d = d
                min_j = j
        route.append(min_j)
        i = min_j
    route.append(start)
    return route

In [672]:
def symmetrize(D):
    return D + D.T - np.diag(D.diagonal())

init_solution = [0, 2, 4, 1, 3, 5, 7, 6, 0]
D = np.array([
    [0, 4, 3, 4, 4, 7, 8, 8],
    [0, 0, 2, 6, 3, 6, 8, 7],
    [0, 0, 0, 4, 1, 4, 6, 5],
    [0, 0, 0, 0, 5, 4, 4, 5],
    [0, 0, 0, 0, 0, 3, 5, 4],
    [0, 0, 0, 0, 0, 0, 3, 2],
    [0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 0]
])
D = symmetrize(D)

In [674]:
best = NN(D, 0)
best, length(D, best)

([0, 2, 4, 1, 3, 5, 7, 6, 0], 28)

### Insertion Heuristics
An intuitive approach to the TSP is to start with a subtour, i.e. a tour on small subsets of nodes, and then extend this tour by inserting the remaining nodes one after the other until all nodes have been inserted. The major difference between insertion schemes is the order in which the nodes are inserted.

In [675]:
def insertion(D):
    route = []
    for i in range(len(D)):
        if i == 0:
            route.append(0)
        elif i == 1:
            route.append(1)
            route.append(0)
        else:
            routes = []
            for j in range(1, len(route)-1):
                temp_route = route.copy()
                temp_route.insert(j, i)
                routes.append(temp_route)
            route = sorted(routes, key=lambda x: length(D, x))[0]
    return route

In [676]:
best = insertion(D)
best, length(D, best)

([0, 3, 6, 7, 5, 4, 2, 1, 0], 21)

## Improvement Heuristics

### The pairwise exchange or 2-opt heuristic
The pairwise exchange or 2-opt technique involves iteratively removing two edges and replacing these with two different edges that reconnect the fragments created by edge removal into a new and shorter tour. This is a special case of the k-opt method. Simple explained: The main idea behind it is to take a route that crosses over itself and reorder it so that it does not.
<img src="https://computingcore.files.wordpress.com/2013/04/kopt.png"/>

#### Example

In [5]:
def pairwise_exchange(route, edges):
    e1, e2 = edges
    return route[:e1+1] + route[e1+1:e2+1][::-1] + route[e2+1:]

In [6]:
route = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'A']
pairwise_exchange(route, [0, 7])

['A', 'H', 'G', 'F', 'E', 'D', 'C', 'B', 'A']

## Chinese Postman Problem
All edges (not nodes) of a network have to be visited. Solving methods include transformation into an asymmetric TSP.

## Cost allocation: Shapley Value (game theory)
The solution, known as the Shapley value, has a nice interpretation in terms of expected marginal contribution. It is calculated by considering all the possible orders of arrival of the players into a game and giving each player his marginal contribution.

1. Calculate worth of 
    - individual players $\{A\},\{B\},\{C\}$
    - their coalitions $S\in\{AB\},\{AC\},\{BC\},\{ABC\}$
2. For all permutations, calculate the marginal contribution of each player
3. For each player, calculate the expected marginal contribution

| Permutation | A | B | C |
|-|-|-|-|
| ABC | v({A}) | v({AB}) - v({A}) | v({ABC}) - v({AB}) |
| ACB | v({A}) | v({ABC}) - v({AC}) | v({AC}) - v({A}) |
| BAC | v({AB}) - v({A}) | v({B}) | v({ABC}) - v({AB}) |
| BCA | v({ABC}) - v({BC}) | v({B}) | v({BC}) - v({B}) |
| CAB | v({AC}) - v({C}) | v({ABC}) - v({AC}) | v({C}) |
| CBA | v({ABC}) - v({BC}) | v({BC}) - v({C}) | v({C}) |

- Good example: http://faculty.econ.ucdavis.edu/faculty/bonanno/teaching/122/Shapley.pdf

# VRP
Extension of TSP by adding capacity constraints.

## Heuristics

### 1. Route, 2. Cluster: TSP and revisit
Determine a tour without capacity constraints (TSP). Then go back to depot if the next customer cannot be visited with given capacity.

1. Optimal TSP solution
    - A => B => C => D => E => F => G => H => A
2. Clustering (length of each route can vary based on capacity constraints)
    - A => B => C => A
    - A => D => E => A
    - A => F => G => A
    - A => H => A

### 1. Cluster, 2. Route: Sweep method
Representation of customers in sequence of their polar coordinates.
<img src="http://images.slideplayer.com/15/4540330/slides/slide_5.jpg" width="500px">

1. Locate all customers on a map or grid
2. Extend a straight line from $(0, 0)$ to first node $(x_1, y_1)$
3. Rotate the line clockwise or counterclockwise
    - Gradually add nodes to the cluster until the vehicle's capacity would be exceeded
    - Calculate the optimal TSP solution for the given cluster
    - Finish the cluster if capacity exceeeded, otherwise continue with the process
4. Repeat the process starting from another node $(x_i, y_i)$
5. Choose the optimal solution 
    - One with the smallest total length
    - Preferably evenly distributed clusters (some buffer for trafic jams, etc.)

### Route + Cluster simultaniously: Savings method
<img src="http://images.slideplayer.com/28/9374696/slides/slide_35.jpg" width="500px">

- Step 1. Savings computation
    - Create ${n}$ vehicle routes ${(0,i,0)}$
        - from depot to customer and back to depot
    - Order the savings in a descreasing fashion.
    - Compute the savings ${s_{ij}=d_{0i}+d_{0j}-d_{ij}}$
        - from depot to customer $n_i$ to customer $n_j$ and back to depot
- Step 2. Starting from the top of the savings list, execute the following:
    - Given a saving ${s_{ij}}$, determine whether there exist two routes that can feasibility be merged:
    - One starting with ${(0,j)}$ and one ending with ${(i,0)}$
        - only connect nodes directly connected to depot
    - Combine these two routes by deleting ${(0,j)}$ and ${(i,0)}$ and introducing ${(i,j)}$.

# Metaheuristics
Heuristics are often problem-dependent, that is, you define an heuristic for a given problem. Meta-heuristics are problem-independent techniques that can be applied to a broad range of problems.

Advantages:
- Finding fast solutions
- General solution procedures

Disadvantages:
- No guarantee of finding the optimum
- No measurement of solution quality

## Simulated Annealing
It is a metaheuristic to approximate global optimization in a large search space. It is often used when the search space is discrete (e.g., all tours that visit a given set of cities). For problems where finding an approximate global optimum is more important than finding a precise local optimum in a fixed amount of time, simulated annealing may be preferable to alternatives such as gradient descent.
<img src="https://image.slidesharecdn.com/part2letterassignment-150624063825-lva1-app6892/95/modelbased-user-interface-optimization-part-ii-letter-assignment-at-sicsa-summer-school-on-computational-interaction-2015-48-638.jpg?cb=1435128014" width="500px">

**Sometimes things really do have to get worse before they can get better.**

- Interactive example: http://toddwschneider.com/posts/traveling-salesman-with-simulated-annealing-r-and-shiny/

In [755]:
def simulated_annealing(D, cooling_rate, stop_it):
    # Create the initial feasable solution. (e.g. NN)
    x = NN(D, 0)
    T = np.sqrt(len(x))
    X = []
    
    for i in range(stop_it):
        # Pick a new candidate tour at random from all neighbors of the existing tour. (e.g. 2-opt randomly)
        e1 = np.random.choice(range(len(x)-3)) # swap of minimum two nodes
        e2 = np.random.choice(range(e1, len(x)-1))
        opt_edges = [e1, e2]
        route = pairwise_exchange(x, opt_edges)
        l = length(D, route)
        delta = l - length(D, x)

        if delta <= 0:
            # If the candidate tour is better than the existing tour, accept it as the new tour
            x = route
            X.append(x)
        else:
            # If the candidate tour is worse than the existing tour, still maybe accept it, according to some probability
            # The probability of accepting an inferior tour is a function of how much longer the candidate is compared to the current tour, and the temperature of the annealing process. 
            # A higher temperature makes you more likely to accept an inferior tour.
            rand = random.uniform(0, 1)
            p = np.exp(-delta/T)
            if rand < p:
                x = route
                X.append(x)
            
        # Cool down the temperature
        T *= cooling_rate
    
    # Return the best solution found so far
    return sorted(X, key=lambda x: length(D, x))[0]

In [756]:
best = simulated_annealing(D, 0.995, 100)
best, length(D, best)

([0, 3, 6, 7, 5, 4, 2, 1, 0], 21)

## Tabu Search
The objective for the Tabu Search algorithm is to constrain from returning to recently visited areas of the search space, referred to as cycling. 
<img src="https://image.slidesharecdn.com/part2letterassignment-150624063825-lva1-app6892/95/modelbased-user-interface-optimization-part-ii-letter-assignment-at-sicsa-summer-school-on-computational-interaction-2015-46-638.jpg?cb=1435128014" width="500px">

In [751]:
def tabu_search(D, nb_size, t_size, stop_it):
    # Generate starting solution (e.g. using NN)
    x = NN(D, 0)
    X = []
    T = []
    
    for i in range(stop_it):
        # - At each step, look for the best solution in the neighborhood
        # - A neighborhood to a given solution is defined as any other solution that is obtained by a pair wise exchange of any two nodes in the solution
        NB = []
        for j in range(nb_size):
            # Generate random opt2-operation
            e1 = np.random.choice(range(len(x)-3)) # swap of minimum two nodes
            e2 = np.random.choice(range(e1, len(x)-1))
            opt_edges = [e1, e2]
            route = pairwise_exchange(x, opt_edges)
            l = length(D, route)
            
            # - To prevent the process from cycling in a small set of solutions, some attribute (pair of exchanged nodes) of recently visited solutions is stored in a Tabu List
            if opt_edges not in T:
                NB.append((opt_edges, route, l))
                
        # Choose the best neighbor
        opt_edges, route, l = sorted(NB, key=lambda n: n[2])[0]
        
        # Update only if new candidate better than the last candidate
        delta = l - length(D, x)
        if delta <= 0:
            x = route
            X.append(x)
            
            # Add the feature of the best candidate to the Tabu list
            T.append(opt_edges)
            
            # Pop first element of Tabu list if maximum size exceeded
            if len(T) > t_size:
                T = T[1:]
    
    # Return the best solution found so far
    return sorted(X, key=lambda x: length(D, x))[0]

In [752]:
best = tabu_search(D, 3, 3, 100)
best, length(D, best)

([0, 3, 6, 7, 5, 4, 2, 1, 0], 21)

## Genetic Algorithm
Genetic algorithms are commonly used to generate high-quality solutions to optimization and search problems by relying on bio-inspired operators such as mutation, crossover and selection. 
<img src="https://nnpdf.hepforge.org/images/GA.png" width="400px">

- GA application on TSP: https://iccl.inf.tu-dresden.de/w/images/b/b7/GA_for_TSP.pdf
- Generic implementation: https://gist.github.com/josephmisiti/940cee03c97f031188ba7eac74d03a4f

In [623]:
def encode_tour(tour):
    # - The mutation operator and the one-point crossover are likely to generate infeasible tours.
    # - The TSP, as opposed to most problems tackled by genetic algorithms, is a pure ordering problem.
    # - Namely, all chromosomes carry exactly the same values and differ only in the ordering of these values.
    # - This implementation focuses on the relative order of the nodes to tacke this problem.
        
    tour = tour.copy()[:-1] # not include last (= first) value
    encoded = []
    # For each element in tour, order list starting from this element and save its index
    for i, node in enumerate(tour):
        encoded.append(sorted(tour[i:]).index(node))
    # Returns list of indices
    return encoded

def subtract(x, y):
    return [item for item in x if item not in y]

def decode_tour(encoded):
    ordered = range(len(encoded))
    decoded = []
    # For each index, find a non-repeating value corresponding to this index
    # from ordered list of all possible values
    for i, node in enumerate(encoded):
        remaining = subtract(range(len(encoded)), decoded)
        decoded.append(remaining[node])
    decoded.append(decoded[0]) # append last (= first) value
    return decoded

In [631]:
def random_population(n, m):
    # - The process begins with a set of individuals which is called a Population. 
    # - An individual is characterized by a set of parameters (variables) known as Genes. 
    # - Genes are joined into a string to form a Chromosome (solution).
    
    # Generate random population with shape (N, M)
    P = []
    for i in range(n):
        ind = list(range(m - 1))
        np.random.shuffle(ind)
        ind.append(ind[0])
        P.append(ind)
    return P

def fitness(D, P):
    # - The fitness of the chromosome is related to the tour length, which in turn depends on the ordering of the nodes.
    # - Since the TSP is a minimization problem, the tour lengths must be transformed so that high fitness values are associated with short tours
    # - A well-known approach is to subtract each tour length to the maximum tour length found in the current population.
    
    values = np.array([length(D, p) for p in P])
    values = np.max(values) - values
    return sorted(zip(P, values), key=lambda x: x[1], reverse=True)

def weighted_select(D, P):
    # - Two pairs of individuals (parents) are selected based on their fitness scores. 
    # - The probability that an individual will be selected for reproduction is based on its fitness score.
    # - Individuals with high fitness have more chance to be selected for reproduction.
    
    F = fitness(D, P)
    _, fitness_values = zip(*F)
    fvsum = np.sum(fitness_values)
    weights = [fv / fvsum for fv in fitness_values]
    indices = np.random.choice(np.arange(0, len(P)), 2, p=weights, replace=False)
    return F[indices[0]][0], F[indices[1]][0]

def crossover(p1, p2):
    # - For each pair of parents to be mated, a crossover point is chosen at random from within the genes.
    # - Offspring are created by exchanging the genes of parents among themselves until the crossover point is reached.
        
    # Generate two new individuals by a random crossover
    cut = np.random.choice(range(1, len(p1)))
    return p1[:cut] + p2[cut:], p2[:cut] + p1[cut:]

def mutate(ind, p):
    # - In certain new offspring formed, some of their genes can be subjected to a mutation with a low random probability.
    # - Mutation occurs to maintain diversity within the population and prevent premature convergence.
        
    # Mutate each gene with certain probability
    mutated = []
    for i in range(len(ind)):
        if np.random.random() <= p:
            # Consider each gen as an index in encoded individual
            # The more to the right, the less an index can be
            allowed_len = len(ind)-len(mutated)
            mutated.append(np.random.choice(range(allowed_len)))
        else:
            mutated.append(ind[i])
    return mutated

In [744]:
def genetic_algorithm(D, n, p, stop_it):
    # Generate population
    P = random_population(n, len(D))
    
    for i in range(stop_it):
        # New generation
        
        # Select
        p1, p2 = weighted_select(D, P)
        
        # Encode
        p1 = encode_tour(p1)
        p2 = encode_tour(p2)
        
        # Crossover
        c1, c2 = crossover(p1, p2)
        
        # Mutate
        c1 = mutate(c1, p)
        c2 = mutate(c2, p)
            
        # Decode
        c1 = decode_tour(c1)
        c2 = decode_tour(c2)
        
        # Add to the Population
        P.append(c1)
        P.append(c2)
        
    # Return the best solution found so far
    return sorted(P, key=lambda p: length(D, p))[0]

In [745]:
best = genetic_algorithm(D, 10, 0.01, 100)
best, length(D, best)

([6, 3, 0, 4, 1, 2, 5, 6], 24)